In [ ]:
import tensorflow as tf
import time
import graph
import coarsening

#import models
%run -n models.ipynb

In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS
# 0.1 for cnn2, 0.3 for fgcnn2, 0.2 for lgcnn2
flags.DEFINE_float('learning_rate', 0.2, 'Initial learning rate.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.')
flags.DEFINE_float('regularization', 0, 'L2 regularizations of weights and biases.')
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('conv_layers', 2, 'Number of convolutional layers.')

# Feature graph

In [ ]:
def grid_graph(m, corners=False):
    z = graph.grid(m)
    A = graph.adjacency(z, k=FLAGS.number_edges)

    # Connections are only vertical or horizontal on the grid.
    # Corner vertices are connected to 2 neightbors only.
    if corners:
        import scipy.sparse
        A = A.toarray()
        A[A < A.max()/1.5] = 0
        A = scipy.sparse.csr_matrix(A)
        print('{} edges'.format(A.nnz))

    print("{} > {} edges".format(A.nnz, FLAGS.number_edges*m**2))
    return A

def coarsen(A, levels):
    graphs, parents = coarsening.metis(A, levels)
    perms = coarsening.compute_perm(parents)

    laplacians = []
    for i,A in enumerate(graphs):
        M, M = A.shape

        # No self-connections.
        if True:
            A = A.tocoo()
            A.setdiag(0)

        if i < FLAGS.conv_layers:
            A = coarsening.perm_adjacency(A, perms[i])

        A = A.tocsr()
        A.eliminate_zeros()
        Mnew, Mnew = A.shape
        print('Layer {0}: M_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz))

        L = graph.laplacian(A, normalized=FLAGS.normalized_laplacian)
        laplacians.append(L)
    return laplacians, perms[0]

t_start = time.process_time()
A = grid_graph(28, corners=False)
L, perm = coarsen(A, FLAGS.conv_layers)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
del A

if False:
    for i,lap in enumerate(L):
        lamb, U = graph.fourier(lap)
        print('L_{}: spectrum in [{:1.2e}, {:1.2e}]'.format(i, lamb[0], lamb[-1]))

# Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data_mnist", one_hot=False)

t_start = time.process_time()
mnist.train._images = coarsening.perm_data(mnist.train._images, perm)
mnist.test._images = coarsening.perm_data(mnist.test._images, perm)
print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
del perm

# Neural network

In [ ]:
model = fc1()
model = fc2(nhiddens=100)
model = cnn2(K=5, F=10)  # K=28 is equivalent to filtering with fgcnn.
model = fcnn2(F=10)
model = fgcnn2(L[0], F=10)
model = lgcnn2_2(L[0], F=10, K=10)
model = cgcnn2_2(L[0], F=10, K=20)
#model = cgcnn2_3(L[0], F=10, K=5)
#model = cgcnn2_4(L[0], F=10, K=5)
#model = cgcnn2_5(L[0], F=10, K=5)

if False:
    K = 5  # 5 or 5^2
    t_start = time.process_time()
    mnist.test._images = graph.lanczos(L, mnist.test._images.T, K).T
    mnist.train._images = graph.lanczos(L, mnist.train._images.T, K).T
    model = lgcnn2_1(L, F=10, K=K)
    print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
    x = tf.placeholder(tf.float32, (None, mnist.train.images.shape[1], K))
else:
    x = tf.placeholder(tf.float32, (None, mnist.train.images.shape[1]))

In [ ]:
#x = tf.placeholder(tf.float32, (FLAGS.batch_size, 784))
#y = tf.placeholder(tf.int32, (FLAGS.batch_size))
#x = tf.placeholder(tf.float32, (None, 784))
y = tf.placeholder(tf.int32, (None))

# Construct computational graph
logits = model.inference(x)
loss = model.loss(logits, y, FLAGS.regularization)
train_op = model.training(loss, FLAGS.learning_rate)
eval_correct = model.evaluation(logits, y)

# Train
t_start = time.process_time()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(int(1e4)):
    batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch_size)
    sess.run(train_op, feed_dict={x: batch_xs, y: batch_ys})
print('Training time: {:.2f}s'.format(time.process_time() - t_start))

# Evaluate
ncorrects = sess.run(eval_correct, feed_dict={x: mnist.test.images, y: mnist.test.labels})
precision = ncorrects / mnist.test.num_examples
print('Precision: {:.2f}% ({:d} / {:d})'.format(precision*100, ncorrects, mnist.test.num_examples))
sess.close()